In [1]:
import utils
import numpy as np

### Helper functions

In [2]:
class coordinates:
    def __init__(self, coords):
        self.row = coords[0]
        self.column = coords[1]

def is_inside_region(coords: coordinates, H: int, W: int):
    """
    Takes in coordindates of candidate pixels, in addition to height and width of region.
    Returns boolean telling if inside region or not
    """
    return coords.column < W and coords.column >= 0 and coords.row < H and coords.row >= 0

def generate_neighbourhood(row, column, H, W):
    """
    Generate Moore-neighbourhood (8-connectedness)
    Returns list of valid neighbours of input in row, column, within given height and width
    """
    # Check for all neighbouring coordinates
    neighbour_matrix = [
            (row - 1, column + 1), (row, column + 1), (row + 1, column + 1), 
            (row - 1, column),                            (row + 1, column), 
            (row - 1, column - 1), (row, column - 1), (row + 1, column - 1)
        ]
    return list(filter(lambda coordinate: is_inside_region(coordinates(coordinate), H, W), neighbour_matrix))

In [3]:
def region_growing(im: np.ndarray, seed_points: list, T: int) -> np.ndarray:
    """
        A region growing algorithm that segments an image into 1 or 0 (True or False).
        Finds candidate pixels with a Moore-neighborhood (8-connectedness). 
        Uses pixel intensity thresholding with the threshold T as the homogeneity criteria.
        The function takes in a grayscale image and outputs a boolean image

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
            seed_points: list of list containing seed points (row, column). Ex:
                [[row1, col1], [row2, col2], ...]
            T: integer value defining the threshold to used for the homogeneity criteria.
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    (H, W) = im.shape
    segmented = np.zeros_like(im).astype(bool)

    for row_seed, column_seed in seed_points:
        current = [(row_seed, column_seed)]
        seed_intensity = im[row_seed, column_seed]
        while len(current) > 0:
            row, column = current.pop(0)
            if not segmented[row, column] and abs((seed_intensity) - (im[row, column])) <= T:
                segmented[row, column] = True
                current += generate_neighbourhood(row, column, H, W)

    return segmented

In [4]:
if __name__ == "__main__":
    # DO NOT CHANGE
    im = utils.read_image("defective-weld.png")

    seed_points = [ # (row, column)
        [254, 138], # Seed point 1
        [253, 296], # Seed point 2
        [233, 436], # Seed point 3
        [232, 417], # Seed point 4
    ]
    # intensity_threshold = 90 # FOR TESTING PURPOSES
    intensity_threshold = 50
    segmented_image = region_growing(im, seed_points, intensity_threshold)

    assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
        im.shape, segmented_image.shape)
    assert segmented_image.dtype == bool, "Expected thresholded image dtype to be np.bool. Was: {}".format(
            segmented_image.dtype)

    segmented_image = utils.to_uint8(segmented_image)
    utils.save_im("defective-weld-segmented.png", segmented_image)

Reading image: img/defective-weld.png
Saving image to: img/processed/defective-weld-segmented.png


/Users/martinjohannesnilsen/OneDrive - NTNU/Fag/Datateknologi 1. semester/TDT4195 - Grunnleggende visuell databehandling/image processing/assignment3/src/utils.py:55: UserWarning: img/processed/defective-weld-segmented.png is a low contrast image
  skimage.io.imsave(impath, im)
